## Strategy

- I have used pre-trained ResNet 18 Model and freezed the FC layers using include_top = False
- I have resized the cifar100 dataset to size 224x224.


In [0]:
from keras.layers import GlobalAveragePooling2D,Activation,Flatten,Conv2D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.preprocessing import image
import keras.backend as K
import cv2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from keras.utils import np_utils
import matplotlib.pyplot as plt
import skimage.transform

In [0]:
pip install image-classifiers

In [0]:
pip install git+https://github.com/qubvel/classification_models.git

  Cloning https://github.com/qubvel/classification_models.git to /tmp/pip-req-build-seflpnoj
  Running command git clone -q https://github.com/qubvel/classification_models.git /tmp/pip-req-build-seflpnoj
  Running command git submodule update --init --recursive -q
  Created wheel for image-classifiers: filename=image_classifiers-1.0.0-cp36-none-any.whl size=19950 sha256=99a9d39f8026ab9e09cd20eec14cbdccb695e23a6cdae6f6c7b1d9d30539362f
  Stored in directory: /tmp/pip-ephem-wheel-cache-1vvebwmk/wheels/de/2b/fd/29a6d33edb8c28bc7d94e95ea1d39c9a218ac500a3cfb1b197
Successfully built image-classifiers


In [0]:
from classification_models.keras import Classifiers 
ResNet18, preprocess_input = Classifiers.get('resnet18') 
#from classification_models.resnet import ResNet18, preprocess_input
model = ResNet18((224, 224, 3), weights='imagenet',include_top=False)

In [0]:
model.summary()

Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, 224, 224, 3)  9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_19 (ZeroPadding2 (None, 230, 230, 3)  0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, 112, 112, 64) 9408        zero_padding2d_19[0][0]          
____________________________________________________________________________________________

In [0]:
num_classes = 100
batch_size = 128
epochs = 5

In [0]:
# add a global spatial average pooling layer
last_layer = model.output
#x = Conv2D(1024, (3,3), strides=(1,1), use_bias=False, border_mode='same')(last_layer)
#x = Conv2D(1024, (3,3), strides=(1,1), use_bias=False, border_mode='same')(x)
#x = Conv2D(1024, (3,3), strides=(1,1), use_bias=False, border_mode='same')(x)
x = Conv2D(num_classes, (1,1), strides=(1,1), use_bias=False)(last_layer)
x = GlobalAveragePooling2D()(x)
# a softmax layer for 10 classes
out = Activation('softmax')(x)

In [0]:
# this is the model we will train
custom_resnet_model = Model(inputs=model.input, outputs=out)

In [0]:
custom_resnet_model.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
data (InputLayer)               (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
bn_data (BatchNormalization)    (None, 224, 224, 3)  9           data[0][0]                       
__________________________________________________________________________________________________
zero_padding2d_19 (ZeroPadding2 (None, 230, 230, 3)  0           bn_data[0][0]                    
__________________________________________________________________________________________________
conv0 (Conv2D)                  (None, 112, 112, 64) 9408        zero_padding2d_19[0][0]          
____________________________________________________________________________________________

In [0]:
custom_resnet_model.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
from keras.datasets import cifar100
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
#x_train = x_train / 255.0
#x_test = x_test / 255.0
y_train = np_utils.to_categorical(y_train, 100)
y_test = np_utils.to_categorical(y_test, 100)

In [0]:
# x_train_mean = np.mean(x_train, axis=(0,1,2))
# x_train_std = np.std(x_train, axis=(0,1,2))
# print(x_train_mean, x_train_std)
# x_test_mean = np.mean(x_test, axis=(0,1,2))
# x_test_std = np.std(x_test, axis=(0,1,2))
# print(x_test_mean, x_test_std)

In [0]:
#x_train.max(), x_train.min(), x_test.max(), x_test.min()

In [0]:
# x_train = x_train.astype('float32')
# x_test = x_test.astype('float32')
#x_train = [(x - x_train_mean) / x_train_std for x in x_train ]
#x_test = [(x - x_test_mean) / x_test_std for x in x_test ]
#x_train = np.asarray(x_train)
#x_test = np.asarray(x_test)
#x_train.max(), x_train.min(), x_test.max(), x_test.min()

In [0]:
#from  skimage import transform
new_shape = (224,224)
x_train = np.asarray([cv2.resize(image, new_shape) for image in x_train])
x_test = np.asarray([cv2.resize(image, new_shape) for image in x_test])

In [0]:
x_test.shape

(10000, 224, 224, 3)

In [0]:
# datagen = ImageDataGenerator(
#    featurewise_center=True,
#    featurewise_std_normalization=True,)
   #horizontal_flip=0.5)
    
#datagen.mean = np.array([129.30416561/255 124.0699627/255  112.43405006/255], dtype=np.float32).reshape((1,1,3)) # ordering: [R, G, B]
#datagen.std = np.array([68.1702429/255  65.39180804/255 70.41837019/255], dtype=np.float32).reshape((1,1,3)) # ordering: [R, G, B]
#datagen.fit(x_train)
    
# train_iterator = datagen.flow(x_train, y_train, batch_size=128, shuffle=True)
    
#datagen_validation = ImageDataGenerator(featurewise_center=True,featurewise_std_normalization=True,)
    
#datagen_validation.mean = np.array([129.74308525/255 124.28521836/255 112.69526387/255], dtype=np.float32).reshape((1,1,3)) # ordering: [R, G, B]
#datagen_validation.std = np.array([68.40415141/255 65.62775279/255 70.65942155/255], dtype=np.float32).reshape((1,1,3)) # ordering: [R, G, B]
#datagen_validation.fit(x_test)
    
#validation_iterator = datagen.flow(x_test, y_test, batch_size=128, shuffle=False)

In [0]:
# history = model.fit_generator(train_iterator,
#                                  steps_per_epoch = len(train_iterator), epochs = 15, 
#                                  validation_data = validation_iterator, 
#                                  validation_steps = len(validation_iterator),
#                                  verbose=1, ) #callbacks=callbacks

In [46]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [0]:
# Specifying the path to store the weights
filepath="/content/gdrive/My Drive/Raajesh-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=True, mode='max')

In [0]:
custom_resnet_model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=15,
              validation_data=(x_test, y_test),
              shuffle=False,
              callbacks=[checkpoint])

Train on 50000 samples, validate on 10000 samples
Epoch 1/15
50000/50000 [==============================] - 397s 8ms/step - loss: 3.4613 - acc: 0.2280 - val_loss: 2.3588 - val_acc: 0.4253

Epoch 00001: val_acc improved from -inf to 0.42530, saving model to /content/gdrive/My Drive/Raajesh-01-0.43.hdf5
Epoch 2/15
50000/50000 [==============================] - 392s 8ms/step - loss: 1.8856 - acc: 0.5320 - val_loss: 1.6119 - val_acc: 0.5758

Epoch 00002: val_acc improved from 0.42530 to 0.57580, saving model to /content/gdrive/My Drive/Raajesh-02-0.58.hdf5
Epoch 3/15
50000/50000 [==============================] - 392s 8ms/step - loss: 1.3558 - acc: 0.6436 - val_loss: 1.3313 - val_acc: 0.6337

Epoch 00003: val_acc improved from 0.57580 to 0.63370, saving model to /content/gdrive/My Drive/Raajesh-03-0.63.hdf5
Epoch 4/15
50000/50000 [==============================] - 391s 8ms/step - loss: 1.0843 - acc: 0.7104 - val_loss: 1.1738 - val_acc: 0.6679

Epoch 00004: val_acc improved from 0.63370 to 